<a href="https://colab.research.google.com/github/sahareslami/Clustering/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview of different clustering algorithms: Clustering methods are used to identify groups of similar objects in multivariate data sets collected. in this project, I use a different clustering algorithm which is implemented in Scikit-learn and Nltk. Scikit-learn is a free software machine learning library for the Python programming language. It features various classification, regression, and clustering algorithms.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from nltk.cluster.kmeans import KMeansClusterer
import nltk
from sklearn import metrics
from sklearn.datasets import load_digits
from sklearn.metrics.cluster import adjusted_rand_score
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift

at the very beginning, I get the data :))

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1Xv9PZ-MYjy2VzrVVkSaZlByUa3w969-d

Downloading...
From: https://drive.google.com/u/0/uc?id=1Xv9PZ-MYjy2VzrVVkSaZlByUa3w969-d
To: /content/Dataset.zip
48.1MB [00:00, 77.6MB/s]


In [ ]:
# !rm -rf "/content/13KjRXkrHg5Mu4m-0KsRXPcHHx2bE4XCZ"
# !rm -rf "/content/13KjRXkrHg5Mu4m-0KsRXPcHHx2bE4XCZ?e=download"
!unzip "/content/Dataset.zip"

Computing accuracy for clustering can be done by rand_index and purity! \
The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings.
The adjusted Rand index is thus ensured to have a value close to 0.0 for random labeling independently of the number of clusters and samples and exactly 1.0 when the clusterings are identical!\
Given the knowledge of the ground truth class assignments labels_true and our clustering algorithm assignments of the same samples labels_pred, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization.
```
sklearn.metrics.adjusted_rand_score(labels_true, labels_pred)[source]¶
```


To compute purity , each cluster is assigned to the class which is most frequent in the cluster, and then the accuracy of this assignment is measured by counting the number of correctly assigned data points and dividing by N.
the function below is an implementation For what is explained above :))

References:\
[Hubert1985
L. Hubert and P. Arabie, Comparing Partitions, Journal of Classification 1985](https://link.springer.com/article/10.1007%2FBF01908075)


In [ ]:
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [ ]:
DataSet = pd.read_csv('/content/test/features.csv' , names = list(range(512)))
vDataSet = pd.read_csv('/content/validation/features.csv' , names = list(range(512)))
vlabels = pd.read_csv('/content/validation/Labels.csv' , names = ['label']).T.to_numpy()
labels = pd.read_csv('/content/test/Labels.csv' , names = ['label']).T.to_numpy()
vlabels = np.resize(vlabels, 332)
labels = np.resize(labels, 2974)

**KMeans**:\
The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares (see below). This algorithm requires the number of clusters to be specified. It scales well to large number of samples and has been used across a large range of application areas in many different fields.

$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$



```
sklearn.cluster.KMeans(n_clusters=8, *, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='deprecated', verbose=0, random_state=None, copy_x=True, n_jobs='deprecated', algorithm='auto')
```

let's do a quick review of parameters that I change them to get better accuracy.\
**init** :\
‘k-means++’ : selects initial cluster centers for k-mean clustering in a smart way to speed up convergence. See section Notes in k_init for more details.

‘random’: choose n_clusters observations (rows) at random from data for the initial centroids.

If an ndarray is passed, it should be of shape (n_clusters, n_features) and gives the initial centers.

If a callable is passed, it should take arguments X, n_clusters and a random state and return an initialization.

**n_init**:\
Number of time the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia.

**max_iter**:\
Maximum number of iterations of the k-means algorithm for a single run.

**random_state**:\
Determines random number generation for centroid initialization. Use an int to make the randomness deterministic


as I mentioned above, having an int number for the random state makes the randomness deterministic! I try 0, 1, and none for this parameter, none gives different accuracy with a maximum of 0.9327302687095453 and the average of 0.7227412651079796 on the validation dataset.\
for a single run with deterministic randomness, I have an accuracy of 0.8174766157769371 for random_state = 0  and 0.8531881082267281 for random_state = 1. \
I also try random seed instead of 'kmeans++' but as I expect I have worse accuracy. (the paper below explain why using kmeans++ is a better idea)\
I try 14 and 15 for the number of clusters! and I have an unusual output! for the test Dataset, I get better accuracy for 14 clusters! it might happen because lack of data for the grand_piano :))\
I also increase the maximum number of iteration per single run (max_iter) and Number of time the k-means algorithm will be run with different centroid seeds(n_init) but nothings get much better!\
I also want to try a different distance measure for Kmeans! I find that it's kind of impossible to do it with Scikit_learn! (HUGE LIBRARY :))) but I find another library(NLTK) that defines slower Kmeans than Scikit_learn but you can choose the type of distance you want to! I try Euclidean and cosine distance and Euclidean have a better accuracy!\
at the end my best rand_index accuracy  is 0.9327302687095453 for validation dataset and 0.8683721143612619 for test dataset and both by running 100 times of default scikit_learn means :)))\

References:
[Mean Shift: A Robust Approach toward Feature Space Analysis](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.76.8968&rep=rep1&type=pdf)



In [ ]:
max_score = 0
avr_score = 0
for i in range(150):
  tkmeans = KMeans(n_clusters=15).fit(DataSet)
  a = adjusted_rand_score(labels, tkmeans.labels_)
  if(a > max_score): 
    max_score = a
  avr_score += a
avr_score /= 150

print(avr_score , max_score)

kmeans = KMeans(n_clusters=14 , random_state= 1 , n_init = 10 , max_iter = 100).fit(DataSet)

# kclusterer = KMeansClusterer(15 , distance=nltk.cluster.util.euclidean_distance, repeats=100)
# assigned_clusters = kclusterer.cluster(DataSet, assign_clusters=True)
# npa = np.asarray(assigned_clusters)

print("rand index evaluation : " , adjusted_rand_score(labels, kmeans.labels_))
print("purity evaluation: "  , purity_score(labels , kmeans.labels_))
# 0.8683721143612619

**Mean shift**:\
MeanShift clustering aims to discover blobs in a smooth density of samples. It is a centroid based algorithm, which works by updating candidates for centroids to be the mean of the points within a given region. These candidates are then filtered in a post-processing stage to eliminate near-duplicates to form the final set of centroids.\
Given a candidate centroid $x_i$ for iteration $t$, the candidate is updated according to the following equation:\
$x_i^{t+1} = m(x_i^t)$\
Where $N(x_i)$ is the neighborhood of samples within a given distance around $x_i$ and $m$ is the mean shift vector that is computed for each centroid that points towards a region of the maximum increase in the density of points. This is computed using the following equation, effectively updating a centroid to be the mean of the samples within its neighborhood:
$m(x_i) = \frac{\sum_{x_j \in N(x_i)}K(x_j - x_i)x_j}{\sum_{x_j \in N(x_i)}K(x_j - x_i)}$

The two most frequently used kernel profiles for the mean shift are flat kernel and Gaussian kernel!
Gaussian kernel is more robust but again scikit_learn implement Mean shift clustering using a flat kernel.\
To calculate the radius, I first tried to get a range for it, then I found the exact number in that range.
The best accuracy was obtained with a radius of 16.3, which is equal to 0.9376508309444408 and 0.9939475453934096 for  rand_index and purity evaluation respectively for test data and  0.8937462153226392 and 0.9759036144578314 for validation data.
**References:**:\
[“Mean shift: A robust approach toward feature space analysis.”](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.76.8968&rep=rep1&type=pdf)\
[Mean Shift, Mode Seeking, and Clustering ](http://home.ku.edu.tr/mehyilmaz/public_html/mean-shift/00400568.pdf)


In [ ]:
bandwith = 0
accuracy = 0
num = np.arange(15.5 , 17.5 , 0.1)

# for i in range (1,40,2):
#   clustering = MeanShift(i).fit(vDataSet)
#   score = adjusted_rand_score(vlabels, clustering.labels_)
#   print(score)

# for i in num:
#   clustering = MeanShift(i).fit(vDataSet)
#   score = adjusted_rand_score(vlabels, clustering.labels_)
#   if(score > accuracy):
#     accuracy = score
#     bandwidth = i

meanShift = MeanShift(16.3).fit(vDataSet)
print("rand index evaluation : " , adjusted_rand_score(vlabels, meanShift.labels_))
print("purity evaluation: "  , purity_score(vlabels , meanShift.labels_))

rand index evaluation :  0.8937462153226392
purity evaluation:  0.9759036144578314


**Agglomerative Clustering**

Hierarchical clustering is a general family of clustering algorithms that build nested clusters by merging or splitting them successively. This hierarchy of clusters is represented as a tree (or dendrogram). The root of the tree is the unique cluster that gathers all the samples, the leaves being the clusters with only one sample.
The AgglomerativeClustering object performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy:\

**Ward** minimizes the sum of squared differences within all clusters. It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach.

**Maximum** or **complete** linkage minimizes the maximum distance between observations of pairs of clusters.

**Average** linkage minimizes the average of the distances between all observations of pairs of clusters.

**Single** linkage minimizes the distance between the closest observations of pairs of clusters


 
![different »inkage](https://doc-0k-5s-docs.googleusercontent.com/docs/securesc/kc1q400hlmlt1l5c0e2qjjelvmpo2scd/mkqfv65k8iet6nhsc1tkc42iauum1ig0/1590698850000/14271364715593122408/14271364715593122408/17iAVzFPMJHU14l6BYEmHfaqjZ_-JQ5kC?e=download&authuser=0&nonce=6fmeq2cmhudlg&user=14271364715593122408&hash=s5p4sah0s4d48ur082srvsd40fp8hr63)
again, let's do a quick review of parameters that I change them to get better accuracy.\

**affinity** 
Metric used to compute the linkage. Can be “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed”. If linkage is “ward”, only “euclidean” is accepted. If “precomputed”, a distance matrix (instead of a similarity matrix) is needed as input for the fit method.

and linkage Which I explained above.

The table below shows a summary of different affinity with different linkage and the rand_index evaluation.\
![alt text](https://doc-04-5s-docs.googleusercontent.com/docs/securesc/kc1q400hlmlt1l5c0e2qjjelvmpo2scd/760tm0adj1bu4ctap0hoa5qgmuoq2615/1590700575000/14271364715593122408/14271364715593122408/1iThc7Toe9Qx7upytl_HMtHRpMkEVWSBd?e=download&authuser=0&nonce=vc4j0tl256ff6&user=14271364715593122408&hash=1en37mikn7rt8g6kj4ajtje2t8iilqgm)


In [ ]:
aagglomerative = AgglomerativeClustering(n_clusters=15 , linkage = 'complete' , affinity = 'cosine').fit(DataSet)
print("rand index evaluation : " , adjusted_rand_score(labels, agglomerative.labels_))
print("purity evaluation: "  , purity_score(labels , agglomerative.labels_))


rand index evaluation :  0.867691465735039
rand index evaluation :  0.9804739294348819
purity evaluation:  0.9690652320107599
